# Objectifs

Dans ce notebook nous allons utiliser notre export JSON des informations collectées sur sur le dépôt arXiv pour produire un modèle sémantique de ces données.
C'est une application du TD3 du module ontologie du mastère mais en changeant la source (et le vecteur) de données.

Nous allons créer une ontologie directement avec owlready2, puis elle sera peuplée.

Dans un 3e temps, je vais tenter d’explorer l’emploi du raisonneur, voire d’autres éléments.

In [1]:
!pip install owlready2


[notice] A new release of pip available: 22.3.1 -> 23.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from owlready2 import *

# Change the following PATH if needed
owlready2.JAVA_EXE=r'c:\Program Files\Java\jdk-19\bin\java.exe'
#owlready2.JAVA_EXE='/usr/bin/java'
allowTheSecondLevel = True # more docs and authors (from ref and citations)

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


In [3]:
filenameDocs = "dbDocs.json"
filenameDocs2 = "dbDocs2.json"
filenameAuthors = "dbAuthors.json"
filenameAuthors2 = "dbAuthors2.json"
filenameAffiliations = "dbAffiliations.json"
filenameTopics = "dbTopics.json"
filenameFields = "dbFields.json"
startDirectory = "D:/mon_depot/"
destination = "../M4.EXP/"

## I. Création de l’ontologie dans PYTHON

### I. Création

In [4]:
#il n'y a pas le logger standard (import logging, maintenant dans python) mais la doc propose :
set_log_level(1)

# Je vais modifier l'ontologie du TD, je la renomme. Elle sera basée sur FOAF puisque HAL l'utilise.
onto = get_ontology('https://cp.org/ProjetFilRouge.owl#')
foaf = get_ontology('foaf.owl').load()


* Owlready2 * Creating new ontology ProjetFilRouge <https://cp.org/ProjetFilRouge.owl#>.
* Owlready2 * Creating new ontology foaf <foaf.owl#>.
* Owlready2 *     ...loading ontology foaf from foaf.owl...
* Owlready2 * WARNING: DataProperty http://xmlns.com/foaf/0.1/nick belongs to more than one entity types: [owl.ObjectProperty, owl.DatatypeProperty]; I'm trying to fix it...
* Owlready2 * WARNING: DataProperty http://xmlns.com/foaf/0.1/aimChatID belongs to more than one entity types: [owl.ObjectProperty, owl.InverseFunctionalProperty, owl.DatatypeProperty, foaf.nick]; I'm trying to fix it...
* Owlready2 * WARNING: DataProperty http://xmlns.com/foaf/0.1/icqChatID belongs to more than one entity types: [owl.ObjectProperty, owl.InverseFunctionalProperty, owl.DatatypeProperty, foaf.nick]; I'm trying to fix it...
* Owlready2 * WARNING: DataProperty http://xmlns.com/foaf/0.1/jabberID belongs to more than one entity types: [owl.ObjectProperty, owl.InverseFunctionalProperty, owl.DatatypePropert

In [5]:
with onto:

    class Topic(Thing): pass
    Topic.comment = ["Topic"]
    class id(Topic >> str, FunctionalProperty): pass
    id.comment = ["S2 topic id"]
    class label(Topic >> str): pass  # No DataProperty in order to vizualise into Neo4j
    label.comment = ["Topic's name"]

    class FieldOfStudy(Thing):pass
    FieldOfStudy.comment = ["Domain of research as defined by arXiv"]
    class arXivCategory(FieldOfStudy >> str,FunctionalProperty): pass  # No DataProperty in order to vizualise into Neo4j
    arXivCategory.comment = ["Domain's name"]

    class Author(foaf.Person): pass
    Author.comment = ["Définition de la classe Personne qui spécialise foaf.Person"]

    class authorId(Author >> str, FunctionalProperty): pass
    authorId.comment = ["S2 id"]
    class aliases(DataProperty):
        domain    = [Author]
        range     = [str]
    aliases.comment = ["Liste d'alias de l'auteur"]
    class fullName(Author >> str, FunctionalProperty): pass  # TODO PRA ??????
    fullName.comment = ["Remplace name, déjà pris par owlready2"]

    class homepage(Author>>str,FunctionalProperty):pass
    homepage.comment = ["Remplace foaf.Homepage qui ne permet pas les url."]

    class Paper(foaf.Document): pass
    Paper.comment = ["Définition de la classe Publication qui spécifie foaf.Document"]
    Paper.comment.append("in fine, comme dans HAL on devrait pouvoir décrire le type d'un document dans arXiv : préprint, these, livre, rapport")
    class arXivId(Paper >> str, FunctionalProperty ) : pass
    arXivId.comment = ["Id ArXiv"]
    class title(DataProperty ) : #pass
        domain    = [Paper]
        range     = [str]
    title.comment = ["Paper's title"]
    class doi(Paper >> str, FunctionalProperty ) : pass
    doi.comment = ["Digital Object Identifier"]
    class paperId(Paper >> str, FunctionalProperty ) : pass
    paperId.comment = ["S2 paper id"]

    class authors(DataProperty):
        domain    = [Paper]
        range     = [str]
    authors.comment = ["S2 Id of an author of the paper"]
    class _citations(DataProperty):
        domain    = [Paper]
        range     = [str]
    _citations.comment = ["S2 Id of a paper citing this paper"]

    class _references(DataProperty):
        domain    = [Paper]
        range     = [str]
    _references.comment = ["S2 Id of a paper referenced by this paper"]

    class title(Paper>>str):pass
    title.comment = ["Title of the paper"]
    class abstract(Paper>>str):pass
    abstract.comment = ["Paper's abstract"]
    class venue(Paper >> str, FunctionalProperty):pass
    venue.comment = ["venue of the paper"]
    class year(Paper >> int, FunctionalProperty):pass
    year.comment = ["year of the paper"]

    class fieldsOfStudy(Paper>>FieldOfStudy):pass
    fieldsOfStudy.comment = ["paper domains"]
    class firstfieldsOfStudy(Paper >> FieldOfStudy, FunctionalProperty):pass

    firstfieldsOfStudy.comment = ["paper main domain"]
    #class language(Paper >> str, FunctionalProperty):pass
    class language(Paper>>str,FunctionalProperty): pass # no DataProperty in order to visualize into Neo4j
    language.comment = ["Language of the paper"]

    class languageProbability(Paper >> float, FunctionalProperty):pass
    languageProbability.comment = ["Computed Probability of Language"]
    class _topics(DataProperty):
        domain    = [Paper]
        range     = [str]
    _topics.comment = ["S2 Id of a topic for this paper"]

    class topic(foaf.topic):
        domaine = [Paper]
        range = [Topic]
    topic.comment = ["Spécialisation de foaf.topic"]

    class reference(Paper>>Paper,ObjectProperty):pass
    reference.comment = ["Paper has target in bibliography"]

    class citationBy(Paper>>Paper,ObjectProperty):
        inverse = reference
    citationBy.comment = ["inverse of reference"]

    class wrote(foaf.publications):
        domaine = [Author]
        range = [Paper]
    wrote.comment = ["spécialisation de foaf.publications"]

    class writtenBy(ObjectProperty):
        inverse = foaf.publications
    writtenBy.comment =["fonction inverse de foaf.publications"]


    class Affiliation(foaf.Organization): pass
    Affiliation.comment = ["Organisation (au sens foaf) d'accueil d'un auteur de publication "]

    class preLabel(Affiliation >> str,FunctionalProperty): pass # DataProperty):
    preLabel.comment = [" Le prefLabel d'une organization"]

    class url(Affiliation >> str,FunctionalProperty): pass   # avant >> str
    url.comment = ["L'url de l'organisazion, presque foaf.homepage mais sans imposer un foaf.Document "]


    class _produced(Affiliation >> str): pass #DataProperty):
    _produced.comment = ["Id d'un article produit par l'organisation. /!\ On perd la date"]

    class produced(Affiliation >> Paper): pass #DataProperty):
    produced.comment = ["Lien vers un article produit par l'organisation. /!\ On perd la date"]

    class _writter(Affiliation>>str): pass #DataProperty):
    _writter.comment = ["id d'un membre qui a publié. /!\ On perd la date"]

    class  member(Affiliation>>Author) : pass #Affiliation >>Author): pass
    member.comment = ["Lien vers les auteurs de l'organisation, spécialisation de foaf"]

    class affiliatedTo(Author>>Affiliation):
        inverse = member
    affiliatedTo.comment =  ["Link to affiliation, inverse of foaf.member"]


Pour tester le raisonneur création d'une propertychain pour reporter les topics des papers vers les auteurs.

In [ ]:
class topic_interest(Author>>Topic,foaf.topic_interest): # foaf.topic_interest
    comment = ["Agregation des topics vers l'auteur"]

topic_interest.property_chain = PropertyChain([wrote, topic])

AllDisjoint([Author,Paper,Affiliation,Topic,FieldOfStudy])

### I.2 Visualisation de l'ontologie dans python

In [6]:
# On peut explorer l'ontologie en python :
for i in onto.classes():  #All classes
    print (i)

ProjetFilRouge.Topic
ProjetFilRouge.FieldOfStudy
ProjetFilRouge.Author
ProjetFilRouge.Paper
ProjetFilRouge.Affiliation


On peut également visualiser plus spécifiquement une propriété

In [7]:
print (foaf.publications.domain,foaf.publications.range)

[foaf.Person] [foaf.Document]


### I.3 Vérification  de l'ontologie dans python
Test rapide du modèle avec un raisonneur.
J'avais un message d'erreur d'owlready2, je vais donc sauver l'ontologie et la tester depuis protégé.

In [8]:
onto.save(destination+"PFR_1.owl")

* Owlready2 * Saving ontology ProjetFilRouge to ../M4.EXP/PFR_1.owl...


Aucune erreur n'est détectée sur ce fichier.

In [9]:
with onto:
    try:
        sync_reasoner(infer_property_values = False)  # par HermiT   infer_property_values = True
        #sync_reasoner(infer_property_values=True, debug=True, keep_tmp_file=True)
        #sync_reasoner_pellet()
        #sync_reasoner_pellet(infer_property_values=True, infer_data_property_values=True, debug=True, keep_tmp_file=True)
        print("Ok, the ontology is consistent.")
    except OwlReadyInconsistentOntologyError:
        print("The ontology is inconsistent!")

* Owlready2 * Saving world <owlready2.namespace.World object at 0x00000145480A8690> to C:\Users\tof\AppData\Local\Temp\tmpbioaf0wl...
* Owlready2 * Running HermiT...
    c:\Program Files\Java\jdk-19\bin\java.exe -Xmx2000M -cp D:\PycharmProjects\arxiv_liv\venv\Lib\site-packages\owlready2\hermit;D:\PycharmProjects\arxiv_liv\venv\Lib\site-packages\owlready2\hermit\HermiT.jar org.semanticweb.HermiT.cli.CommandLine -c -O -D -I file:///C:/Users/tof/AppData/Local/Temp/tmpbioaf0wl


Ok, the ontology is consistent.


* Owlready2 * HermiT took 0.771085262298584 seconds
* Owlready * Reparenting ProjetFilRouge.topic_interest: {foaf.topic_interest, owl.ObjectProperty} => {foaf.topic_interest}
* Owlready * Reparenting foaf.primaryTopic: {owl.FunctionalProperty, owl.ObjectProperty} => {owl.FunctionalProperty, foaf.topic}
* Owlready * Reparenting foaf.img: {foaf.depiction, owl.ObjectProperty} => {foaf.depiction}
* Owlready * Reparenting foaf.homepage: {owl.InverseFunctionalProperty, foaf.isPrimaryTopicOf, owl.ObjectProperty, foaf.page} => {foaf.isPrimaryTopicOf}
* Owlready * Reparenting foaf.openid: {owl.InverseFunctionalProperty, foaf.isPrimaryTopicOf, owl.ObjectProperty} => {foaf.isPrimaryTopicOf}
* Owlready * Reparenting foaf.icqChatID: {foaf.nick, owl.DatatypeProperty} => {foaf.nick}
* Owlready * Reparenting foaf.aimChatID: {foaf.nick, owl.DatatypeProperty} => {foaf.nick}
* Owlready * Reparenting foaf.yahooChatID: {foaf.nick, owl.DatatypeProperty} => {foaf.nick}
* Owlready * Reparenting foaf.msnChatID

L'ontologie est bien consistante.

## II. Peuplement de l'ontologie

# Import des domaines de recherche

In [10]:
%%time
import json
dicoFieldOfStudy = {}
with open(startDirectory + filenameFields, encoding="utf8") as f:
    for line in f:
        current = json.loads(line)
        for item in current:
            domain = FieldOfStudy(namespace=onto,arXivCategory=item)
            dicoFieldOfStudy[item] = domain
            #print(domain.get_iri())

        #aff = Affiliation(namespace=onto,preLabel=current["Name"])

CPU times: total: 0 ns
Wall time: 5.01 ms


In [11]:
onto.search_one(type = onto.FieldOfStudy, iri = "https://cp.org/ProjetFilRouge.owl#fieldofstudy32")

ProjetFilRouge.fieldofstudy32

In [12]:
def searchFieldsOfStudy(name):
    #return onto.search_one(type = onto.FieldOfStudy, arXivCategory = str(name))
    return dicoFieldOfStudy[name]

In [13]:
#for i in FieldOfStudy.instances():
#    print(i.label)

# import des auteurs de premier niveau

In [14]:
dicoAuthor = {}
def readAuthors(filename):
    try:
        with open(startDirectory + filename, encoding="utf8") as f:
            for line in f:
                #print(line)
                current = json.loads(line)
                aut = Author(namespace=onto,authorId = current["authorId"],fullName = current["name"])
                dicoAuthor[current["authorId"]]=aut
                #https://www.semanticscholar.org/author/145099042
                aka =current.get("aliases",[])
                if len(aka) > 0:
                    aka2 = "; ".join(aka)  # TO vizualise into Neo4J
                    aut.aliases = [aka2]
                hp =current.get("homepage","")
                if hp is not None and len(hp)>0:
                    aut.homepage = hp
    except IOError:
        print ("Erreur! Le fichier n'a pas pu être ouvert")

In [15]:
%%time
readAuthors(filenameAuthors)


CPU times: total: 1.09 s
Wall time: 1.69 s


Pour économiser de la mémoire, je désactive la création des auteurs de second niveau

In [16]:
%%time
if False and allowTheSecondLevel:
    readAuthors(filenameAuthors2)

CPU times: total: 0 ns
Wall time: 0 ns


In [17]:
#for i in onto.individuals():
#    print (i,i.is_a)
#    for prop in i.get_properties():
#        for value in prop[i]:
#            print(".%s == %s" % (prop.python_name, value))

# import des documents

In [18]:
dicoPaper = {}
def readDocs(filename):
    try:
        with open(startDirectory + filename, encoding="utf8") as f:
            for line in f:
                current = json.loads(line)
                #print(line)
                doc = Paper(namespace=onto,paperId =current["paperId"],title=[current["title"]])
                dicoPaper[current["paperId"]]=doc  # test !
                #print(current["paperId"])
                #https://www.semanticscholar.org/paper/7349311ca0bc34989bf1d27da1b5a28d2ec9e1e4
                #doc.set_iri( )  # TODO ?
                #title =current.get("title","")
                #if len(title) > 0:
                #    doc.arXivId = arxivId

                arxivId =current.get("arxivId","")
                if len(arxivId) > 0:
                    doc.arXivId = arxivId
                abstract =current.get("abstract","")
                if len(abstract) > 0:
                    doc.abstract = [abstract]
                doi =current.get("doi","")
                if doi is not None and len(doi) > 0:
                    doc.doi = doi
                citations =current.get("citations",[])
                if len(citations) > 0:
                    doc._citations = citations
                references =current.get("references",[])
                if len(references) > 0:
                    doc._references = references
                authors =current.get("authors",[])
                if len(authors) > 0:
                    la = [a for a in authors if a is not None]
                    if len(la)>0:
                        doc.authors = la
                topics =current.get("topics",[])
                if len(topics) > 0:
                    doc._topics = topics
                venue =current.get("venue","")
                if len(venue) > 0:
                    doc.venue = venue
                year =current.get("year",0)
                if year is not None and year > 0:
                    doc.year = year
                fieldsOfStudy =current.get("fieldsOfStudy",[])
                if len(fieldsOfStudy) > 0:
                    #print("f:",fieldsOfStudy)
                    l = [searchFieldsOfStudy(i) for i in fieldsOfStudy]
                    #print(l)
                    doc.fieldsOfStudy = l
                    doc.firstfieldsOfStudy = l[0]

                Language =current.get("Language","")
                if len(Language) > 0:
                    #print("**")
                    doc.language = Language
                LanguageProbability =current.get("LanguageProbability",0.)
                if LanguageProbability > 0.:
                    #print("^^")
                    doc.languageProbability = LanguageProbability
    except IOError:
        print( "Erreur! Le fichier n'a pas pu être ouvert")

In [19]:
%%time
readDocs(filenameDocs)

CPU times: total: 8.38 s
Wall time: 11 s


In [20]:
#for i in Paper.instances():
#    print(i.languageProbability)

In [21]:
%%time
if allowTheSecondLevel:
    readDocs(filenameDocs2)

CPU times: total: 1min 10s
Wall time: 1min 40s


# import des affiliations

In [22]:
%%time
import json

with open(startDirectory + filenameAffiliations, encoding="utf8") as f:
    for line in f:
        current = json.loads(line)
        d = current["year"]
        la = set()
        lp = set()
        for v in d.items():
            #print(v)
            for aut in v[1]["authors"]:
                la.add(aut)
            for pap in v[1]["papers"]:
                lp.add(pap)
        aff = Affiliation(namespace=onto,preLabel=current["Name"])
        #print(la,lp)
        aff._writter = [*la]
        aff._produced = [*lp]


CPU times: total: 188 ms
Wall time: 297 ms


# Création des topics

In [23]:
%%time
dicoTopic = {}
with open(startDirectory + filenameTopics, encoding="utf8") as f:
    for line in f:
        current = json.loads(line)

        for k,v in current.items():
            #print(k,v)
            t = Topic(namespace=onto,id=k,label=[v])
            dicoTopic[k] = t
            # https://www.semanticscholar.org/topic/Social-network-analysis/33270

CPU times: total: 766 ms
Wall time: 979 ms


# Création des liens
(j'ai tenté d'utiliser les property_chain pour réaliser cette tache par le raisoner, mais
c'est très peu documenté. Je fais donc les liens à la main.

In [24]:
def searchAuthor(id):
    return dicoAuthor.get(id,None)

def searchPaper(id):
    return dicoPaper.get(id)

def searchTopic(id):
    return dicoTopic[id]

In [25]:
%%time

for aff in Affiliation.instances():
    #print(aff)
    for ii in aff._writter:
        aut = searchAuthor(ii)
        if  aut is not None:
            #print("%%%")
            aff.member.append(aut)
            aut.affiliatedTo.append(aff) # Pour eviter le passage par Protégé

    aff._writter = []
    for ii in aff._produced:
        #print(ii)
        pap = searchPaper(ii)
        if pap is not None:
            aff.produced.append(pap)
            #print("**")
    aff._produced = []
    #print(str(i.writter()))

CPU times: total: 203 ms
Wall time: 299 ms


In [26]:
%%time
for pap in Paper.instances():
    for ii in pap._references:
        pap2 = searchPaper(ii)
        if pap2 is not None:
            pap.reference.append(pap2)
    pap._references = []
    for ii in pap._citations:
        pap2 = searchPaper(ii)
        if pap2 is not None:
            #pap.citationBy.append(pap2)
            pap2.reference.append(pap)
    pap._citations = []

    for ii in pap._topics:
        #print ("iiii=",ii)
        top = searchTopic(ii)
        if top is not None:
            #print("*")
            pap.topic.append(top)
    pap._topics = []
    for ii in pap.authors:
        aut = searchAuthor(ii)
        if aut is not None:
            #aut.publications.append(pap)
            aut.wrote.append(pap)
    pap.authors = []

CPU times: total: 1min 8s
Wall time: 1min 49s


In [27]:
%%time
onto.save(destination+"PFR_2.owl")

* Owlready2 * Saving ontology ProjetFilRouge to ../M4.EXP/PFR_2.owl...


CPU times: total: 12.8 s
Wall time: 31.7 s


In [28]:
with onto:
    try:
        #sync_reasoner(infer_property_values = True)  # par HermiT   infer_property_values = True
        sync_reasoner(infer_property_values=True, debug=True, keep_tmp_file=True)
        #sync_reasoner_pellet()
        #sync_reasoner_pellet(infer_property_values=True, infer_data_property_values=True, debug=True, keep_tmp_file=True)
        print("Ok, the ontology is consistent.")
    except OwlReadyInconsistentOntologyError:
        print("The ontology is inconsistent!")

* Owlready2 * Saving world <owlready2.namespace.World object at 0x00000145480A8690> to C:\Users\tof\AppData\Local\Temp\tmpahrlzis2...
* Owlready2 * Running HermiT...
    c:\Program Files\Java\jdk-19\bin\java.exe -Xmx2000M -cp D:\PycharmProjects\arxiv_liv\venv\Lib\site-packages\owlready2\hermit;D:\PycharmProjects\arxiv_liv\venv\Lib\site-packages\owlready2\hermit\HermiT.jar org.semanticweb.HermiT.cli.CommandLine -c -O -D -I file:///C:/Users/tof/AppData/Local/Temp/tmpahrlzis2 -Y


OwlReadyJavaError: Java error message is:
Exception in thread "main" java.lang.OutOfMemoryError: Java heap space
	at org.semanticweb.HermiT.model.InterningManager.createEntries(Unknown Source)
	at org.semanticweb.HermiT.model.InterningManager.intern(Unknown Source)
	at org.semanticweb.HermiT.model.Constant.create(Unknown Source)
	at org.semanticweb.HermiT.structural.OWLClausification$DataRangeConverter.visit(Unknown Source)
	at uk.ac.manchester.cs.owl.owlapi.OWLLiteralImplNoCompression.accept(OWLLiteralImplNoCompression.java:281)
	at org.semanticweb.HermiT.structural.OWLClausification$FactClausifier.visit(Unknown Source)
	at uk.ac.manchester.cs.owl.owlapi.OWLDataPropertyAssertionAxiomImpl.accept(OWLDataPropertyAssertionAxiomImpl.java:97)
	at org.semanticweb.HermiT.structural.OWLClausification.clausify(Unknown Source)
	at org.semanticweb.HermiT.structural.OWLClausification.preprocessAndClausify(Unknown Source)
	at org.semanticweb.HermiT.Reasoner.loadOntology(Reasoner.java:117)

Les propriétés chainées (propagation des topics) est bien inférée.

In [29]:
with onto:
    try:
        #sync_reasoner(infer_property_values = True)  # par HermiT   infer_property_values = True
        sync_reasoner(infer_property_values=True, debug=True, keep_tmp_file=True)
        #sync_reasoner_pellet()
        #sync_reasoner_pellet(infer_property_values=True, infer_data_property_values=True, debug=True, keep_tmp_file=True)
        print("Ok, the ontology is consistent.")
    except OwlReadyInconsistentOntologyError:
        print("The ontology is inconsistent!")

* Owlready2 * Saving world <owlready2.namespace.World object at 0x00000145480A8690> to C:\Users\tof\AppData\Local\Temp\tmpz50lje3u...
* Owlready2 * Running HermiT...
    c:\Program Files\Java\jdk-19\bin\java.exe -Xmx2000M -cp D:\PycharmProjects\arxiv_liv\venv\Lib\site-packages\owlready2\hermit;D:\PycharmProjects\arxiv_liv\venv\Lib\site-packages\owlready2\hermit\HermiT.jar org.semanticweb.HermiT.cli.CommandLine -c -O -D -I file:///C:/Users/tof/AppData/Local/Temp/tmpz50lje3u -Y


OwlReadyJavaError: Java error message is:
Exception in thread "main" java.lang.OutOfMemoryError: Java heap space
	at org.semanticweb.HermiT.model.InterningManager.createEntries(Unknown Source)
	at org.semanticweb.HermiT.model.InterningManager.intern(Unknown Source)
	at org.semanticweb.HermiT.model.Constant.create(Unknown Source)
	at org.semanticweb.HermiT.structural.OWLClausification$DataRangeConverter.visit(Unknown Source)
	at uk.ac.manchester.cs.owl.owlapi.OWLLiteralImplNoCompression.accept(OWLLiteralImplNoCompression.java:281)
	at org.semanticweb.HermiT.structural.OWLClausification$FactClausifier.visit(Unknown Source)
	at uk.ac.manchester.cs.owl.owlapi.OWLDataPropertyAssertionAxiomImpl.accept(OWLDataPropertyAssertionAxiomImpl.java:97)
	at org.semanticweb.HermiT.structural.OWLClausification.clausify(Unknown Source)
	at org.semanticweb.HermiT.structural.OWLClausification.preprocessAndClausify(Unknown Source)
	at org.semanticweb.HermiT.Reasoner.loadOntology(Reasoner.java:117)

Etrangement, Protégé infère plus, mais un warning indique qu'une partie seulement des inference sont montrées.
On peut observer dans protégé que les relations inverses sont bien trouvées.


In [ ]:
onto.save(destination+"PFR_3.owl")